In [ ]:
import os
import re
import sys
import glob
import pickle
from copy import deepcopy
from collections import OrderedDict

import numpy as np
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_slide
from dlml.nn import predict

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = [3]
rec_bus_IDs = [3, 14, 17]
additional_tags = ['ReLU_none', 'PowerFactory']

experiments = collect_experiments(area_ID, area_measure=area_measure, D=None, DZA=None, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=None, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_idx = np.argmin(val_loss)
best_checkpoint = checkpoint_files[epochs.index(best_idx + 1)]
model = keras.models.load_model(best_checkpoint, compile=True)
data_dirs = ['..' + os.path.sep +
             os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d])
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names']]
var_names_fmt = list(OrderedDict({k + '{}': [] for k in tmp}).keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('G 01', 5.00), ('G 02', 4.33), ('G 03', 4.47), ('G 04', 3.57), ('G 05', 4.33),
    ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
])


if 'area1_wrong' in tags:
    generators_areas_map = {
        'default': [
            ['G 02', 'G 03', 'G 10'],
            ['G 04', 'G 05', 'G 06', 'G 07'],
            ['G 08', 'G 09'],
            ['G 01']
        ]
    }
else:
    generators_areas_map = {
        'default': [
            ['G 02', 'G 03'],
            ['G 04', 'G 05', 'G 06', 'G 07'],
            ['G 08', 'G 09', 'G 10'],
            ['G 01']
        ]
    }

P_nom = {'G 01': 10000e6, 'G 02': 700e6, 'G 03': 800e6, 'G 04':  800e6, 'G 05':  300e6,
         'G 06':   800e6, 'G 07': 700e6, 'G 08': 700e6, 'G 09': 1000e6, 'G 10': 1000e6}

window_dur = 60
window_step = 1

var_names_network = network_parameters['var_names']
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

In [ ]:
S2 = 700e6
S3 = 800e6
M = 0.2
H2 = 4
H3 = (30 * M * 1e9 - H2 * S2) / S3

In [ ]:
S2 = 700e6
S3a = 800e6
S3b = 200e6
M = 0.23
H2 = 4.33
H3a = 2.5
H3b = (30 * M * 1e9 - H2 * S2 - H3a * S3a) / S3b
print(H3b)
# H3a, H3b = 4.47, 4.47
print((H2 * S2 + H3a * S3a + H3b * S3b) / 30 * 1e-9)

In [ ]:
mom = lambda H2,H3: (H2 * S2 + H3 * S3) / 30 * 1e-9
mom(4, 4)
mom(4, 3.6)

In [ ]:
generators_areas_map

## Constant measure

In [ ]:
H_values = [
    default_H,
    OrderedDict([
        ('G 01', 5.00), ('G 02', 5.00), ('G 03', 3.884), ('G 04', 3.57), ('G 05', 4.33),
        ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
    ]),
    OrderedDict([
        ('G 01', 5.00), ('G 02', 5.00), ('G 03', 4.00), ('G 04', 3.57), ('G 05', 4.33),
        ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.107)
    ])
]
N_H = len(H_values)

measure_exact = []

data = []
data_normalized = []
measure = []
area_inertia = []

var_names_data = [
    'Pe_line_3_4', 'Qe_line_3_4', 'Vd_bus3', 'Vq_bus3',
    'Pe_line_14_15', 'Qe_line_14_15', 'Vd_bus14', 'Vq_bus14',
    'Pe_line_16_17', 'Qe_line_16_17', 'Vd_bus17', 'Vq_bus17',
]
var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

replace_keys = lambda D, keys_in, keys_out: {key_out: D[key_in] for key_in, key_out in zip(keys_in, keys_out)}

for H in H_values:
    data_file = data_dir + '/IEEE39_rnd_load_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names_data,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_norm = replace_keys(data_norm, var_names_data, var_names_network)
    data_sliding = replace_keys(data_sliding, var_names_data, var_names_network)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(8, n_vars * 2.5))
gs = fig.add_gridspec(n_vars+1, 4)
ax = []
for i in range(n_vars):
    ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])]),
# ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:])])
ax.append([fig.add_subplot(gs[-1,:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

bus_ID = rec_bus_IDs[0]

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
if dm == 0:
    dm = np.max(measure_exact) / 10
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        key = var_name.format(bus_ID)
        value = np.squeeze(data_normalized[i][key])
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][0].plot(time / 60, measure[i], 'k', lw=1)
    ax[-1][0].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), 'k--', lw=1)
    ax[-1][0].set_ylim(ylim)
    ax[-1][0].set_xlabel('Time [min]')

for row in ax:
    for side in 'right','top':
        for a in row:
            a.spines[side].set_visible(False)

ax[0][0].legend(loc='best')
# ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][1])

ax[-1][0].set_xlim(time[[0,-1]] / 60)
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
# output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
#     f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
# fig.savefig(output_filename)

## With generator #3 (in area 1) split in two

In [ ]:
if 'area1_wrong' in tags:
    generators_areas_map['split_gen'] = [
        ['G 02', 'G 03a', 'G 03b', 'G 10'],
        ['G 04', 'G 05', 'G 06', 'G 07'],
        ['G 08', 'G 09'],
        ['G 01']
    ]
    H_values = [
        default_H,
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 7), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 7), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 7), ('G 03b', 7), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ])
    ]
else:
    generators_areas_map['split_gen'] = [
        ['G 02', 'G 03a', 'G 03b'],
        ['G 04', 'G 05', 'G 06', 'G 07'],
        ['G 08', 'G 09', 'G 10'],
        ['G 01']
    ]
    H_values = [
        default_H,
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 2.5), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 2.5), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ])
    ]

P_nom['G 03a'], P_nom['G 03b'] = 600e6, 200e6

N_H = len(H_values)

data = []
data_normalized = []
measure = []
measure_exact = []
area_inertia = []

for H in H_values:
    if len(H) == 10:
        data_file = data_dir + '/IEEE39_rnd_load_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    else:
        data_file = data_dir + '/IEEE39_rnd_load_split_gen_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '_{:.0f}_{:.0f}.h5' \
        .format(P_nom['G 03a']*1e-6, P_nom['G 03b']*1e-6)
        _,_,v,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, 'inertia')

    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, d, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data.append(d)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
idx, = np.where(np.array(list(map(len, H_values))) == 10)
jdx, = np.where(np.array(list(map(len, H_values))) == 11)
ms = 8
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(measure_exact[idx], measure_predicted[idx], 'ko', ms=ms, markerfacecolor='w', markeredgewidth=2)
ax.plot(measure_exact[jdx], measure_predicted[jdx], 'rs', ms=ms, markerfacecolor='w', markeredgewidth=2)
ax.plot(measure_exact[[0,-1]], measure_exact[[0,-1]], '--', lw=1, color=[.6,.6,.6])
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel(f'{area_measure.capitalize()} [{measure_units}]')
ax.set_ylabel(f'Predicted {area_measure} [{measure_units}]')
ax.grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
# ax.set_xlim([0.358, 0.392])
# ax.set_ylim([0.348, 0.392])
# ax.set_xticks(np.r_[0.36 : 0.39 : 0.01])
# ax.set_yticks(np.r_[0.35 : 0.39 : 0.01])
fig.tight_layout()

## With generator #3 (in area 1) split in two and different values of nominal power

In [ ]:
if 'area1_wrong' in tags:
    generators_areas_map['split_gen'] = [
        ['G 02', 'G 03a', 'G 03b', 'G 10'],
        ['G 04', 'G 05', 'G 06', 'G 07'],
        ['G 08', 'G 09'],
        ['G 01']
    ]
    H_values = [
        default_H,
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 7), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 7), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 7), ('G 03b', 7), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 7), ('G 03b', 7), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ])
    ]
else:
    generators_areas_map['split_gen'] = [
        ['G 02', 'G 03a', 'G 03b'],
        ['G 04', 'G 05', 'G 06', 'G 07'],
        ['G 08', 'G 09', 'G 10'],
        ['G 01']
    ]
    H_values = [
        default_H,
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 4.47), ('G 03b', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 0.345), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 1.845), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 3.345), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 4.845), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 6.345), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
        OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03a', 2.5), ('G 03b', 7.845), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ]),
    ]


P_nom['G 03a'], P_nom['G 03b'] = 800e6, 200e6

N_H = len(H_values)

data = []
data_normalized = []
measure = []
measure_exact = []
area_inertia = []

for H in H_values:
    if len(H) == 10:
        data_file = data_dir + '/IEEE39_rnd_load_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    elif 'area_1_wrong' in tags and H != H_values[-1]:
        data_file = data_dir + '/IEEE39_rnd_load_split_gen_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '_{:.0f}_{:.0f}.h5' \
        .format(P_nom['G 03a']*1e-6, P_nom['G 03b']*1e-6)
        _,_,v,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, 'inertia')
    else:
        data_file = data_dir + '/IEEE39_rnd_load_split_gen_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '_{:.0f}_{:.0f}_Q0.h5' \
        .format(P_nom['G 03a']*1e-6, P_nom['G 03b']*1e-6)
        _,_,v,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['split_gen'], P_nom, 'inertia')

    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, d, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data.append(d)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
idx, = np.where(np.array(list(map(len, H_values))) == 10)
jdx, = np.where(np.array(list(map(len, H_values))) == 11)
ms = 8
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(measure_exact[idx], measure_predicted[idx], 'ko', ms=ms, markerfacecolor='w', markeredgewidth=2)
ax.plot(measure_exact[jdx], measure_predicted[jdx], 'rs', ms=ms, markerfacecolor='w', markeredgewidth=2)
ax.plot([measure_exact.min(), measure_exact.max()],
        [measure_exact.min(), measure_exact.max()], '--', lw=1, color=[.6,.6,.6])
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel(f'{area_measure.capitalize()} [{measure_units}]')
ax.set_ylabel(f'Predicted {area_measure} [{measure_units}]')
ax.grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
# ax.set_xlim([0.358, 0.392])
# ax.set_ylim([0.348, 0.392])
# ax.set_xticks(np.r_[0.36 : 0.39 : 0.01])
# ax.set_yticks(np.r_[0.35 : 0.39 : 0.01])
fig.tight_layout()
output_filename = f'IEEE39_PF_area{area_ID}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_compensator_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)

In [ ]:
output_filename

## With a 4th generator (a compensator) added in area 1

In [ ]:
bus = 8

if bus == 8:
    compensator_name = 'G 11'
    P_nom[compensator_name] = 500e6
else:
    compensator_name = 'Comp 01'
    P_nom[compensator_name] = 100e6

generators_areas_map['compensator'] = deepcopy(generators_areas_map['default'])
generators_areas_map['compensator'][area_ID - 1].append(compensator_name)

H_compensator = np.concatenate([[0.1, 0.2, 0.5], np.r_[1 : 10]])

H_values = [
    default_H
]
for h in H_compensator:
    if bus == 8:
        tmp = OrderedDict([
            ('G 01', 5.00), ('G 02', 4.33), ('G 03', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20),
            (compensator_name, h)
        ])
    else:
        tmp = OrderedDict([(compensator_name, h),
            ('G 01', 5.00), ('G 02', 4.33), ('G 03', 4.47), ('G 04', 3.57), ('G 05', 4.33),
            ('G 06', 4.35), ('G 07', 3.77), ('G 08', 3.47), ('G 09', 3.45), ('G 10', 4.20)
        ])
    H_values.append(tmp)
N_H = len(H_values)

data_normalized = []
measure = []
measure_exact = []
area_inertia = []

zero_Q = True
data_subdir = f'/with_compensator/bus_{bus}'
if zero_Q:
    data_subdir += '/zero_Q'
else:
    data_subdir += '/nonzero_Q'

for H in H_values:
    if compensator_name in H:
        data_file = data_dir + data_subdir + '/IEEE39_w__Stoch._Load_&_Compensator_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['compensator'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['compensator'], P_nom, 'inertia')
    else:
        data_file = data_dir + '/IEEE39_w__Stoch._Load_' + \
            '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
        _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')

    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)

In [ ]:
measure_predicted = np.array(list(map(np.nanmean, measure)))
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(measure_exact[[1,-1]], measure_exact[[1,-1]], 'k--', lw=1)
ax.plot(measure_exact[1:], measure_predicted[1:], 'ko', markerfacecolor='w', markersize=5, markeredgewidth=1)
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel(f'{area_measure.capitalize()} [{measure_units}]')
ax.set_ylabel(f'Predicted {area_measure} [{measure_units}]')
ax.grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
# ax.set_xlim([0.358, 0.392])
# ax.set_ylim([0.348, 0.392])
# ax.set_xticks(np.r_[0.36 : 0.39 : 0.01])
# ax.set_yticks(np.r_[0.35 : 0.39 : 0.01])
fig.tight_layout()
fig.savefig(data_dir + data_subdir + '/compensator.pdf')

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(9, n_vars * 3))
gs = fig.add_gridspec(n_vars+1, 4)
ax = []
for i in range(n_vars):
    ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])])
ax.append([fig.add_subplot(gs[-1, :])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0], [0,0,.7]]
cmap = plt.get_cmap('gray', N_H+1)
bus_ID = rec_bus_IDs[0]

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        key = var_name.format(bus_ID)
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=cmap(i), lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=cmap(i), lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][0].plot(time / 60, measure[i], color=cmap(i), lw=1)
    ax[-1][0].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), '--', color=cmap(i), lw=1)
    ax[-1][0].set_ylim(ylim)
    ax[-1][0].set_xlabel('Time [min]')

for aa in ax:
    for side in 'right','top':
        for a in aa:
            a.spines[side].set_visible(False)
for i in range(n_vars):
    for j in range(2):
        ax[i][j].grid(axis='y', lw=0.5, linestyle=':')

# ax[0][0].legend(loc='best')
ax[-1][0].set_xlim([0,30])
# ax[-1][0].set_ylim([0.3, 0.4])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
# output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
#     f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_compensator_{experiment_ID[:6]}.pdf'
# fig.savefig(output_filename)

In [ ]:
# mean absolute percentage error
mape = np.array([np.nanmean(np.abs((m - me) / me)) * 100 for m,me in zip(measure, measure_exact)])
# mean absolute error
mae = np.array([np.nanmean(np.abs(m - me)) for m,me in zip(measure, measure_exact)])
# increase in area momentum when the compensator is present
Dm = (measure_exact - measure_exact[0]) / measure_exact[0] * 100
# mean prediction
measure_predicted = np.array([np.nanmean(m) for m in measure])

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})

rows = 2
cols = 1
x_offset = [0.17, 0.03]
y_offset = [0.17, 0.075]
x_space = 0.1
y_space = 0.175

fig = plt.figure(figsize=(8/2.54, 6/2.54))
ax = make_axes(rows, cols, x_offset, y_offset, x_space, y_space)

ax[0].plot(measure_exact[[0,-1]], measure_exact[[0,-1]], 'k--', lw=1)
ax[0].plot(measure_exact[0], measure_predicted[0], 'ko', markerfacecolor='k', markersize=4, markeredgewidth=1)
ax[0].plot(measure_exact[1:], measure_predicted[1:], 'ko', markerfacecolor='w', markersize=4, markeredgewidth=1)
dx = measure_exact[-1] - measure_exact[0]
ax[0].set_xlim([measure_exact[0] - dx/20, measure_exact[-1] + dx/20])
ax[0].set_xlabel(f'{area_measure[0].capitalize()} [{measure_units}]')
ax[0].set_xticks(np.r_[0.35 : 0.5 : 0.05])
ax[0].set_ylim([measure_exact[0] - dx/5, measure_exact[-1] + dx/10])
ax[0].set_ylabel(f'Predicted {area_measure[0].capitalize()} [{measure_units}]')
ax[0].set_yticks(np.r_[0.35 : 0.55 : 0.05])

start = 4
x = Dm[start:] - Dm[start]
y = mape[start:] - mape[start]
power_law = lambda x,a,b: a * (x ** b)
popt,pcov = curve_fit(power_law, x, y, p0=[1,0.5])
x = np.linspace(x[0], x[-1], 50)
y = power_law(x, *popt) + mape[start]
x += Dm[start]
ax[1].plot(x, y, color=[.7,.7,.7], lw=2)
ax[1].plot(Dm[[1,start-1]], mape[1:start].mean() + np.zeros(2), color=[.4,.4,.4], lw=2)
ax[1].plot(Dm[0], mape[0], 'ko', markerfacecolor='k', markeredgewidth=1, markersize=4)
ax[1].plot(Dm[1:], mape[1:], 'ko', markerfacecolor='w', markeredgewidth=1, markersize=4)
dx = Dm[-1] - Dm[0]
ax[1].set_xlim([Dm[0] - dx/20, Dm[-1] + dx/20])
ax[1].set_ylim([0,33])
ax[1].set_yticks(np.r_[0:35:10])
ax[1].set_xlabel(f'{area_measure[0].capitalize()} increase [%]')
ax[1].set_ylabel('MAPE [%]')

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='y', lw=0.5, color=[.6,.6,.6], ls=':')
    
output_filename = f'IEEE39_PF_area{area_ID}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_compensator_error_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)